In [18]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np


In [2]:


weather = pd.read_csv('Data_fixed.csv', index_col = "Tanggal")

output = weather.apply(pd.isnull).sum()/weather.shape[0]

core_weather = weather[["Tavg", "RH_avg","RR", "ss", "ff_avg"]].copy()

core_weather.columns = ["Temp_rata-rata", "Kelembapan_rata-rata","Curah-hujan","Lamanya-penyinaran-matahari","durasi_hujan"]



print(output)

Tn         0.003650
Tx         0.000000
Tavg       0.000000
RH_avg     0.000000
RR         0.003650
ss         0.007299
ff_x       0.000000
ddd_x      0.000000
ff_avg     0.000000
ddd_car    0.000000
dtype: float64


In [3]:
core_weather = core_weather.dropna()  # Untuk menghapus baris yang mengandung nilai-nilai hilang
# atau
core_weather = core_weather.fillna(core_weather.mean())  # Menggantikan nilai-nilai hilang dengan rata-rata

In [4]:
for col in core_weather.columns:
    min = core_weather[col].min()
    max = core_weather[col].max()
    core_weather[col +'-scaled'] = (core_weather[col] - min) / ( max - min )
core_weather

,Temp_rata-rata,Kelembapan_rata-rata,Curah-hujan,Lamanya-penyinaran-matahari,durasi_hujan,Temp_rata-rata-scaled,Kelembapan_rata-rata-scaled,Curah-hujan-scaled,Lamanya-penyinaran-matahari-scaled,durasi_hujan-scaled
Tanggal,,,,,,,,,,
1/1/2023,26.4,90,55.1,0.0,2,0.360656,0.787879,0.006199,0.000000,0.333333
2/1/2023,26.4,90,55.1,0.0,2,0.360656,0.787879,0.006199,0.000000,0.333333
3/1/2023,26.4,90,55.1,0.0,2,0.360656,0.787879,0.006199,0.000000,0.333333
4/1/2023,26.0,92,4.1,0.0,2,0.295082,0.848485,0.000461,0.000000,0.333333
5/1/2023,25.6,90,18.6,0.0,3,0.229508,0.787879,0.002093,0.000000,0.666667
...,...,...,...,...,...,...,...,...,...,...
27-09-2023,29.4,65,1.5,9.2,3,0.852459,0.030303,0.000169,0.876190,0.666667
28-09-2023,29.7,65,1.5,10.0,3,0.901639,0.030303,0.000169,0.952381,0.666667
29-09-2023,30.3,66,1.5,9.9,3,1.000000,0.060606,0.000169,0.942857,0.666667


In [5]:
# menguji pengaruh kelembapan rata rata, curah hujan, lamanya penyinaran matahari, kecepatan angin rata rata terhadap suhu

x_uji1 = sm.add_constant(core_weather[["Kelembapan_rata-rata-scaled","Curah-hujan-scaled","Lamanya-penyinaran-matahari-scaled","durasi_hujan-scaled"]])
y_uji1 = core_weather["Temp_rata-rata-scaled"]

model1 = sm.OLS(y_uji1,x_uji1).fit()

print (model1.summary())




                              OLS Regression Results                             
Dep. Variable:     Temp_rata-rata-scaled   R-squared:                       0.493
Model:                               OLS   Adj. R-squared:                  0.486
Method:                    Least Squares   F-statistic:                     64.73
Date:                   Mon, 11 Dec 2023   Prob (F-statistic):           3.63e-38
Time:                           10:20:47   Log-Likelihood:                 203.02
No. Observations:                    271   AIC:                            -396.0
Df Residuals:                        266   BIC:                            -378.0
Df Model:                              4                                         
Covariance Type:               nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

In [ ]:
def membuat_model_prediksi_suhu(a, b, c, d):
    """
    parameter a adalah parameter untuk nilai kelembapan
    parameter b adalah parametet untuk nilai curah hujan
    parameter c adalah parameter untuk lamanya penyinaran matahari
    parameter d adalah parameter untuk durasi hujan
    constant adalah nilai  ketika dari seluruh parameter bernilai 0
    """
    coef_kelembapan = -0.4089
    coef_curah_hujan = -0.0603
    coef_lamanya_penyinaran_matahari = 0.1233
    coef_durasi_hujan = -0.0700
    constant = 0.6894

    return a*coef_kelembapan + b * coef_curah_hujan + c * coef_lamanya_penyinaran_matahari + d * coef_durasi_hujan + constant


In [6]:
# menguji pengaruh kelembapan rata rata, curah hujan, lamanya penyinaran matahari, kecepatan angin rata rata terhadap suhu

x_uji2 = sm.add_constant(core_weather[["Temp_rata-rata-scaled","Curah-hujan-scaled","Lamanya-penyinaran-matahari-scaled","durasi_hujan-scaled"]])
y_uji2 = core_weather["Kelembapan_rata-rata-scaled"]

model2 = sm.OLS(y_uji2,x_uji2).fit()

print (model2.summary())




                                 OLS Regression Results                                
Dep. Variable:     Kelembapan_rata-rata-scaled   R-squared:                       0.602
Model:                                     OLS   Adj. R-squared:                  0.596
Method:                          Least Squares   F-statistic:                     100.4
Date:                         Mon, 11 Dec 2023   Prob (F-statistic):           5.72e-52
Time:                                 10:20:47   Log-Likelihood:                 177.87
No. Observations:                          271   AIC:                            -345.7
Df Residuals:                              266   BIC:                            -327.7
Df Model:                                    4                                         
Covariance Type:                     nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
-----------------

In [10]:
def membuat_model_prediksi_kelembapan(a, b, c, d):
    """
    parameter a adalah parameter untuk nilai suhu
    parameter b adalah parametet untuk nilai curah hujan
    parameter c adalah parameter untuk lamanya penyinaran matahari
    parameter d adalah parameter untuk durasi hujan
    constant adalah nilai  ketika dari seluruh parameter bernilai 0
    """
    coef_suhu = -0.4922
    coef_curah_hujan = -0.0429
    coef_lamanya_penyinaran_matahari = -0.3283
    coef_durasi_hujan = 0.0120
    constant = 0.9066

    return a*coef_suhu + b * coef_curah_hujan + c * coef_lamanya_penyinaran_matahari + d * coef_durasi_hujan + constant


In [7]:
# menguji pengaruh kelembapan rata rata, curah hujan, lamanya penyinaran matahari, kecepatan angin rata rata terhadap suhu

x_uji3 = sm.add_constant(core_weather[["Temp_rata-rata-scaled","Kelembapan_rata-rata-scaled","Lamanya-penyinaran-matahari-scaled","durasi_hujan-scaled"]])
y_uji3 = core_weather["Curah-hujan-scaled"]

model3 = sm.OLS(y_uji3,x_uji3).fit()

print (model3.summary())




                            OLS Regression Results                            
Dep. Variable:     Curah-hujan-scaled   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     7.083
Date:                Mon, 11 Dec 2023   Prob (F-statistic):           1.97e-05
Time:                        10:20:47   Log-Likelihood:                -86.893
No. Observations:                 271   AIC:                             183.8
Df Residuals:                     266   BIC:                             201.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
cons

In [11]:
def membuat_model_prediksi_curah_hujan(a, b, c, d):
    """
    parameter a adalah parameter untuk nilai suhu
    parameter b adalah parametet untuk nilai kelembapan
    parameter c adalah parameter untuk lamanya penyinaran matahari
    parameter d adalah parameter untuk durasi hujan
    constant adalah nilai  ketika dari seluruh parameter bernilai 0
    """
    coef_suhu = -0.5121
    coef_kelembapan = -0.3028
    coef_lamanya_penyinaran_matahari = 0.2075
    coef_durasi_hujan = -0.2186
    constant = 0.5509

    return a*coef_suhu + b * coef_kelembapan + c * coef_lamanya_penyinaran_matahari + d * coef_durasi_hujan + constant


In [13]:
x_uji4 = sm.add_constant(core_weather[["Temp_rata-rata-scaled","Kelembapan_rata-rata-scaled","Curah-hujan-scaled","durasi_hujan-scaled"]])
y_uji4 = core_weather["Lamanya-penyinaran-matahari-scaled"]

model4 = sm.OLS(y_uji4,x_uji4).fit()

print (model4.summary())


                                    OLS Regression Results                                    
Dep. Variable:     Lamanya-penyinaran-matahari-scaled   R-squared:                       0.521
Model:                                            OLS   Adj. R-squared:                  0.514
Method:                                 Least Squares   F-statistic:                     72.33
Date:                                Mon, 11 Dec 2023   Prob (F-statistic):           2.15e-41
Time:                                        10:38:38   Log-Likelihood:                 65.259
No. Observations:                                 271   AIC:                            -120.5
Df Residuals:                                     266   BIC:                            -102.5
Df Model:                                           4                                         
Covariance Type:                            nonrobust                                         
                                  coef    std err 

In [14]:
def membuat_model_prediksi_curah_hujan(a, b, c, d):
    """
    parameter a adalah parameter untuk nilai suhu
    parameter b adalah parametet untuk nilai kelembapan
    parameter c adalah parameter untuk curah hujan
    parameter d adalah parameter untuk durasi hujan
    constant adalah nilai  ketika dari seluruh parameter bernilai 0
    """
    coef_suhu = 0.3409
    coef_kelembapan = -0.7537
    coef_curah_hujan = 0.0675
    coef_durasi_hujan = -0.0258
    constant = 0.6169

    return a*coef_suhu + b * coef_kelembapan + c * coef_curah_hujan + d * coef_durasi_hujan + constant


In [15]:
x_uji5 = sm.add_constant(core_weather[["Temp_rata-rata-scaled","Kelembapan_rata-rata-scaled","Curah-hujan-scaled","Lamanya-penyinaran-matahari-scaled"]])
y_uji5 = core_weather["durasi_hujan-scaled"]

model5 = sm.OLS(y_uji5,x_uji5).fit()

print (model5.summary())


                             OLS Regression Results                            
Dep. Variable:     durasi_hujan-scaled   R-squared:                       0.130
Model:                             OLS   Adj. R-squared:                  0.117
Method:                  Least Squares   F-statistic:                     9.964
Date:                 Mon, 11 Dec 2023   Prob (F-statistic):           1.58e-07
Time:                         10:40:36   Log-Likelihood:                -46.802
No. Observations:                  271   AIC:                             103.6
Df Residuals:                      266   BIC:                             121.6
Df Model:                            4                                         
Covariance Type:             nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [ ]:
def membuat_model_prediksi_curah_hujan(a, b, c, d):
    """
    parameter a adalah parameter untuk nilai suhu
    parameter b adalah parametet untuk nilai kelembapan
    parameter c adalah parameter untuk curah hujan
    parameter d adalah parameter untuk nilai lamanya penyinaran matahari
    constant adalah nilai  ketika dari seluruh parameter bernilai 0
    """
    coef_suhu = -0.4426
    coef_kelembapan = 0.0632
    coef_curah_hujan = -0.1626
    coef_lamanya_penyinaran_matahari = -0.0590
    constant = 0.5514

    return a*coef_suhu + b * coef_kelembapan + c * coef_curah_hujan + d * coef_lamanya_penyinaran_matahari + constant


In [48]:
# split data dengan sklearn train test split
# split data untuk prediksi suhu
x1 = x_uji1
y1 = y_uji1
X1_train, X1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.2, random_state=42)

# split data untuk prediksi kelembapan
x2 = x_uji2
y2 = y_uji2
X2_train, X2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.2, random_state=42)

# split data untuk prediksi curah hujan
x3 = x_uji3
y3 = y_uji3
X3_train, X3_test, y3_train, y3_test = train_test_split(x3, y3, test_size=0.2, random_state=42)

# split data untuk prediksi lamanya penyinaran matahari

x4 = x_uji4
y4 = y_uji4
X4_train, X4_test, y4_train, y4_test = train_test_split(x4, y4, test_size=0.2, random_state=42)

# split data untuk prediksi durasi hujan

x5 = x_uji5
y5 = y_uji5
X5_train, X5_test, y5_train, y5_test = train_test_split(x5, y5, test_size=0.2, random_state=42)


In [43]:
prediksi = LinearRegression()

prediksi.fit(X1_train, y1_train)

prediksi_suhu = prediksi.predict(X1_test)

prediksi_suhu

array([0.33123612, 0.34035464, 0.31619904, 0.42712296, 0.54679163,
       0.42568189, 0.54973993, 0.33123612, 0.45669802, 0.64033223,
       0.55737467, 0.63383422, 0.56806046, 0.31619904, 0.32547893,
       0.47248723, 0.56806046, 0.42067475, 0.50658878, 0.31619904,
       0.63095855, 0.75732351, 0.33123612, 0.33123612, 0.55737467,
       0.4223408 , 0.56511919, 0.56220383, 0.50658878, 0.56511919,
       0.50658878, 0.31619904, 0.54917118, 0.56806046, 0.59442452,
       0.38240065, 0.38049472, 0.31619904, 0.38049472, 0.38049472,
       0.56511919, 0.37229123, 0.76946836, 0.56511919, 0.31619904,
       0.59468722, 0.40198952, 0.5182578 , 0.50077301, 0.55737467,
       0.56511919, 0.58295203, 0.62079338, 0.6300381 , 0.46873093])

In [51]:
prediksi.fit(X2_train, y2_train)
prediksi_kelembapan = prediksi.predict(X2_test)
prediksi_kelembapan


array([0.73543312, 0.62628877, 0.74895675, 0.53752753, 0.53094872,
       0.55421856, 0.53583134, 0.73543312, 0.63196732, 0.2246515 ,
       0.43250305, 0.20996087, 0.55716777, 0.74895675, 0.74061717,
       0.45187443, 0.55716777, 0.50031589, 0.34001105, 0.74895675,
       0.56470144, 0.16171939, 0.73543312, 0.73543312, 0.43250305,
       0.56806087, 0.49417894, 0.5330181 , 0.34001105, 0.49417894,
       0.34001105, 0.74895675, 0.48969396, 0.55716777, 0.19525732,
       0.61691024, 0.53683276, 0.74895675, 0.53683276, 0.53683276,
       0.49417894, 0.65915615, 0.32864526, 0.49417894, 0.74895675,
       0.5083184 , 0.6255844 , 0.56992309, 0.52887626, 0.43250305,
       0.49417894, 0.39483134, 0.53931005, 0.31429555, 0.54509039])

In [53]:
prediksi.fit(X3_train, y3_train)
prediksi_curah_hujan = prediksi.predict(X3_test)
prediksi_curah_hujan

array([ 0.07254287,  0.15647758, -0.04138827,  0.01726397,  0.28020381,
        0.15082786,  0.27557941,  0.07254287,  0.11900655,  0.06616878,
        0.21104512,  0.16615135,  0.24631108, -0.04138827,  0.38395071,
        0.24756822,  0.24631108,  0.15342935,  0.22162423, -0.04138827,
        0.3566918 ,  0.11110496,  0.07254287,  0.07254287,  0.21104512,
        0.16060436,  0.22865816,  0.30014798,  0.22162423,  0.22865816,
        0.22162423, -0.04138827,  0.07897088,  0.24631108,  0.04599194,
        0.19017312,  0.08488232, -0.04138827,  0.08488232,  0.08488232,
        0.22865816,  0.02005047,  0.24731325,  0.22865816, -0.04138827,
        0.20978797,  0.01562005,  0.14797446,  0.24049745,  0.21104512,
        0.22865816,  0.18063505,  0.33980364,  0.13039504,  0.17856768])

In [54]:
prediksi.fit(X4_train, y4_train)
prediksi_lamanya_penyinaran_matahari = prediksi.predict(X4_test)
prediksi_lamanya_penyinaran_matahari

array([ 0.08205281,  0.03064342,  0.11333895,  0.31843945,  0.5927502 ,
        0.21289641,  0.59037673,  0.08205281,  0.3081329 ,  0.67496399,
        0.43906438,  0.59243963,  0.44423185,  0.11333895, -0.16103561,
        0.43740677,  0.44423185,  0.4040758 ,  0.50069983,  0.11333895,
        0.51326727,  0.89082913,  0.08205281,  0.08205281,  0.43906438,
        0.17359439,  0.44062481,  0.61199469,  0.50069983,  0.44062481,
        0.50069983,  0.11333895,  0.52160192,  0.44423185,  0.58251203,
        0.06943888,  0.16032096,  0.11333895,  0.16032096,  0.16032096,
        0.44062481,  0.20159446,  0.85430833,  0.44062481,  0.11333895,
        0.52398486,  0.26597626,  0.62648276,  0.29265889,  0.43906438,
        0.44062481,  0.51365939,  0.49918078,  0.63091543,  0.28615661])

In [55]:
prediksi.fit(X5_train, y5_train)
prediksi_durasi_hujan = prediksi.predict(X5_test)
prediksi_durasi_hujan

array([0.47514912, 0.45636736, 0.47187113, 0.29474973, 0.17328978,
       0.37557167, 0.19341041, 0.47514912, 0.38692964, 0.12803064,
       0.2847562 , 0.15995721, 0.36201278, 0.47187113, 0.63011156,
       0.17611464, 0.36201278, 0.17281323, 0.12568899, 0.47187113,
       0.36297614, 0.0575953 , 0.47514912, 0.47514912, 0.2847562 ,
       0.43324465, 0.33270316, 0.17526904, 0.12568899, 0.33270316,
       0.12568899, 0.47187113, 0.25843382, 0.36201278, 0.13624366,
       0.49138505, 0.36184365, 0.47187113, 0.36184365, 0.36184365,
       0.33270316, 0.39462503, 0.15831394, 0.33270316, 0.47187113,
       0.30847657, 0.36183248, 0.15376226, 0.39219885, 0.2847562 ,
       0.33270316, 0.23901022, 0.35159763, 0.17721752, 0.36044285])

In [34]:
# Ambil kolom suhu dari core_weather
suhu_column = core_weather['Temp_rata-rata']

# Hitung nilai minimum dan maksimum dari kolom suhu
min_suhu = suhu_column.min()
max_suhu = suhu_column.max()

# Buat DataFrame untuk menyimpan hasil inversi scaling
prediksi_suhu_baru = pd.DataFrame()

# Lakukan inversi scaling hanya untuk kolom suhu
# for col in prediksi_suhu:
#     prediksi_suhu_baru[col] = (prediksi_suhu[col]) * (max_suhu - min_suhu) + min_suhu

prediksi_suhu_skala_asli = (prediksi_suhu * (max_suhu - min_suhu)) + min_suhu

print(prediksi_suhu_skala_asli)


[26.22054036 26.27616333 26.12881413 26.80545004 27.53542897 26.79665956
 27.55341358 26.22054036 26.98585794 28.10602658 27.59998548 28.06638877
 27.66516878 26.12881413 26.18542147 27.08217208 27.66516878 26.766116
 27.29019157 26.12881413 28.04884713 28.81967339 26.22054036 26.22054036
 27.59998548 26.77627888 27.64722708 27.62944335 27.29019157 27.64722708
 27.29019157 26.12881413 27.54994418 27.66516878 27.82598958 26.53264396
 26.5210178  26.12881413 26.5210178  26.5210178  27.64722708 26.4709765
 28.89375701 27.64722708 26.12881413 27.82759206 26.65213605 27.36137259
 27.25471533 27.59998548 27.64722708 27.7560074  27.98683959 28.0432324
 27.05925867]


In [57]:
# Ambil kolom suhu dari core_weather
kelembapan = core_weather['Kelembapan_rata-rata']

# Hitung nilai minimum dan maksimum dari kolom suhu
min_kelembapan = kelembapan.min()
max_kelembapan = kelembapan.max()

# Buat DataFrame untuk menyimpan hasil inversi scaling

# Lakukan inversi scaling hanya untuk kolom suhu
# for col in prediksi_suhu:
#     prediksi_suhu_baru[col] = (prediksi_suhu[col]) * (max_suhu - min_suhu) + min_suhu

prediksi_kelembapan_skala_asli = (prediksi_kelembapan * (max_kelembapan - min_kelembapan)) + min_kelembapan

print(prediksi_kelembapan_skala_asli)


[88.26929309 84.66752956 88.7155727  81.73840855 81.52130783 82.28921247
 81.68243427 88.26929309 84.85492155 71.41349958 78.27260078 70.92870875
 82.38653625 88.7155727  88.44036677 78.91185633 82.38653625 80.51042446
 75.22036468 88.7155727  82.63514753 69.33673995 88.26929309 88.26929309
 78.27260078 82.74600884 80.30790506 81.58959745 75.22036468 80.30790506
 75.22036468 88.7155727  80.15990065 82.38653625 70.44349147 84.35803788
 81.71548116 88.7155727  81.71548116 81.71548116 80.30790506 85.75215307
 74.84529367 80.30790506 88.7155727  80.77450721 84.64428518 82.80746203
 81.45291643 78.27260078 80.30790506 77.0294342  81.79723169 74.3717533
 81.98798295]


In [68]:
# Ambil kolom suhu dari core_weather
curah_hujan = core_weather['Curah-hujan']

# Hitung nilai minimum dan maksimum dari kolom suhu
min_curah_hujan = curah_hujan.min()
max_curah_hujan = curah_hujan.max()


prediksi_curah_hujan_skala_asli = (prediksi_curah_hujan * (max_curah_hujan - min_curah_hujan)) + min_curah_hujan

print(prediksi_curah_hujan_skala_asli)


[ 644.7610024  1390.7727019  -367.85897349  153.44220475 2490.45149831
 1340.55804723 2449.3498032   644.7610024  1057.73020672  588.10812081
 1875.76900392 1476.75322033 2189.21285832 -367.85897349 3412.55389023
 2200.38635282 2189.21285832 1363.68002084 1969.79613085 -367.85897349
 3170.27670318  987.50091562  644.7610024   644.7610024  1875.76900392
 1427.4515362  2032.31370896 2667.71526194 1969.79613085 2032.31370896
 1969.79613085 -367.85897349  701.89314888 2189.21285832  408.77633768
 1690.25866908  754.43408491 -367.85897349  754.43408491  754.43408491
 2032.31370896  178.20860445 2198.12020942 2032.31370896 -367.85897349
 1864.59550942  138.8310222  1315.1969636  2137.54136376 1875.76900392
 2032.31370896 1605.48432645 3020.1747697  1158.95110957 1587.10957147]


In [62]:
# Ambil kolom suhu dari core_weather
lamanya_penyinaran_matahari = core_weather['Lamanya-penyinaran-matahari']

# Hitung nilai minimum dan maksimum dari kolom suhu
min_lamanya_penyinaran_matahari = lamanya_penyinaran_matahari.min()
max_lamanya_penyinaran_matahari = lamanya_penyinaran_matahari.max()


prediksi_lamanya_penyinaran_matahari_skala_asli = (prediksi_lamanya_penyinaran_matahari * (max_lamanya_penyinaran_matahari - min_lamanya_penyinaran_matahari)) + min_lamanya_penyinaran_matahari

print(prediksi_lamanya_penyinaran_matahari_skala_asli)


[ 0.86155446  0.32175591  1.19005901  3.34361422  6.22387715  2.23541235
  6.19895563  0.86155446  3.2353955   7.08712191  4.61017598  6.22061613
  4.6644344   1.19005901 -1.69087392  4.59277112  4.6644344   4.24279594
  5.25734817  1.19005901  5.38930629  9.35370583  0.86155446  0.86155446
  4.61017598  1.82274107  4.62656045  6.42594426  5.25734817  4.62656045
  5.25734817  1.19005901  5.47682014  4.6644344   6.11637629  0.72910827
  1.68337004  1.19005901  1.68337004  1.68337004  4.62656045  2.11674181
  8.97023745  4.62656045  1.19005901  5.50184103  2.79275073  6.57806897
  3.07291835  4.61017598  4.62656045  5.39342356  5.24139822  6.62461198
  3.0046444 ]


In [64]:
# Ambil kolom suhu dari core_weather
durasi_hujan = core_weather['durasi_hujan']

# Hitung nilai minimum dan maksimum dari kolom suhu
min_durasi_hujan = durasi_hujan.min()
max_durasi_hujan = durasi_hujan.max()


prediksi_durasi_hujan_skala_asli = (prediksi_durasi_hujan * (max_durasi_hujan - min_durasi_hujan)) + min_durasi_hujan

print(prediksi_durasi_hujan_skala_asli)


[2.42544736 2.36910209 2.41561339 1.8842492  1.51986933 2.12671502
 1.58023122 2.42544736 2.16078892 1.38409193 1.85426859 1.47987164
 2.08603834 2.41561339 2.89033469 1.52834392 2.08603834 1.51843969
 1.37706697 2.41561339 2.08892841 1.1727859  2.42544736 2.42544736
 1.85426859 2.29973396 1.99810947 1.52580713 1.37706697 1.99810947
 1.37706697 2.41561339 1.77530146 2.08603834 1.40873099 2.47415515
 2.08553094 2.41561339 2.08553094 2.08553094 1.99810947 2.18387508
 1.47494182 1.99810947 2.41561339 1.92542972 2.08549744 1.46128678
 2.17659656 1.85426859 1.99810947 1.71703065 2.05479288 1.53165257
 2.08132854]


In [38]:
# evaluasi model

y1_test_skala_asli = (y1_test * (max_suhu - min_suhu)) + min_suhu


mse1 = mean_squared_error(y1_test_skala_asli, prediksi_suhu_skala_asli)
rmse1 = np.sqrt(mse1)

print(f"Mean Squared Error: {mse1}")
print(f"Root Mean Squared Error: {rmse1}")



Mean Squared Error: 0.6064583321652595
Root Mean Squared Error: 0.7787543464824189


In [77]:


y2_test_skala_asli = (y2_test * (max_kelembapan - min_kelembapan)) + min_kelembapan


mse2 = mean_squared_error(y2_test_skala_asli, prediksi_kelembapan_skala_asli)
rmse2 = np.sqrt(mse2)

print(f"Mean Squared Error: {mse2}")
print(f"Root Mean Squared Error: {rmse2}")


Mean Squared Error: 25.349224130110386
Root Mean Squared Error: 5.034801299963126


In [73]:


y3_test_skala_asli = (y3_test * (max_curah_hujan - min_curah_hujan)) + min_curah_hujan


mse3 = mean_squared_error(y3_test_skala_asli, prediksi_curah_hujan_skala_asli)
rmse3 = np.sqrt(mse3)

print(f"Mean Squared Error: {mse3}")
print(f"Root Mean Squared Error: {rmse3}")


Mean Squared Error: 9750012.039774934
Root Mean Squared Error: 3122.5009271055364


In [74]:


y4_test_skala_asli = (y4_test * (max_lamanya_penyinaran_matahari - min_lamanya_penyinaran_matahari)) + min_lamanya_penyinaran_matahari


mse4 = mean_squared_error(y4_test_skala_asli, prediksi_lamanya_penyinaran_matahari_skala_asli)
rmse4 = np.sqrt(mse4)

print(f"Mean Squared Error: {mse4}")
print(f"Root Mean Squared Error: {rmse4}")


Mean Squared Error: 4.548355127706781
Root Mean Squared Error: 2.132687301904989


In [75]:

y5_test_skala_asli = (y5_test * (max_durasi_hujan - min_durasi_hujan)) + min_durasi_hujan


mse5 = mean_squared_error(y5_test_skala_asli, prediksi_lamanya_penyinaran_matahari_skala_asli)
rmse5 = np.sqrt(mse5)

print(f"Mean Squared Error: {mse5}")
print(f"Root Mean Squared Error: {rmse5}")


Mean Squared Error: 10.245566747443869
Root Mean Squared Error: 3.2008696861078034
